### Basic Vector and Matrix operations

In [1]:
import numpy as np
import torch

In [2]:
torch.manual_seed(42)

#### Transpose

In [3]:
# torch.arange(start, stop, step) creates a vector whose elements go from
# start to stop in increments of step. E.g., torch.arange(0, 72, 8) will 
# be [0, 8, 16, 24, ..64]. We will create an image with 4 rows and 9 cols
# using this function now.
I49 = torch.stack([torch.arange(0, 72, 8), torch.arange(64, 136, 8),
                torch.arange(128, 200, 8), torch.arange(192, 264, 8)])
print("Shape of the matrix is: {}".format(I49.shape))

# Transpose of a matrix interchanges rows and cols. A 4 x 9 matrix
# becomes 9 x 4 on transposition.
I49_t = torch.transpose(I49, 0, 1)
print("Shape of transposed matrix is: {}".format(I49_t.shape))

# Let us asssert that it is a true transpose, i.e., I[i][j] == I_t[j][1]
for i in range(0, I49.shape[0]):
    for j in range(0, I49.shape[1]):
        assert I49[i][j] == I49_t[j][i]

# .T retrieves the transpose of a matrix (array)
assert torch.allclose(I49_t, I49.T, 1e-5)

Shape of the matrix is: torch.Size([4, 9])
Shape of transposed matrix is: torch.Size([9, 4])


#### Dot product
The dot product of two vectors $\vec{a}$ and $\vec{b}$ represents the
component of one vector along the other.

Consider two vectors $\vec{a} = [a_1\;\;a_2\;\;a_3]$ and
$\vec{b} = [b_1\;\;b_2\;\;b_3]$.
<br>Then $\vec{a}\space.\vec{b} = a_1b_1 + a_2b_2 + a_3b_3$

In [4]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
a_dot_b = torch.dot(a, b)
print("Dot product of these two vectors is: "
      "{}".format(a_dot_b))

# Dot product of perpendicular vectors is zero
vx = torch.tensor([1, 0]) # a vector along X-axis
vy = torch.tensor([0, 1]) # a vector along Y-axis
print("Example dot product of orthogonal vectors:"
      " {}".format(torch.dot(vx, vy)))

Dot product of these two vectors is: 32
Example dot product of orthogonal vectors: 0


#### Matrix multiplication
Matrices can be multiplied with other matrices or vector.

##### Matrix vector multiplication
Consider a matrix $A_{m,n}$ with m rows and n columns which is multiplied
with a vector $\vec{b_{n}}$ with n elements.

Below we show an example with $m = 3$ and $n = 2$.

The resultant vector $\vec{c_{m}}$ is:
\begin{align*}
\begin{bmatrix}
        c_{1} \\
        c_{2}  \\
        c_{3}
\end{bmatrix}
& = 
\begin{bmatrix}
        a_{11} & a_{12} \\
        a_{21} & a_{22} \\
        a_{31} & a_{32}
\end{bmatrix}
\begin{bmatrix}
        b_{1} \\
        b_{2} \\
\end{bmatrix}\\
\\
c_{1} &= a_{11}b_{1} + a_{12}b_{2} \\
c_{2} &= a_{21}b_{2} + a_{22}b_{2} \\
c_{3} &= a_{31}b_{2} + a_{32}b_{2}
\end{align*}

In general
$$
c_{i} = a_{i1}b_{1} + a_{i2}b_{2} + \cdots + a_{in}b_{n} \\
$$

In [5]:
# Let us consider the familiar cat-brain training dataset
# We have defined our model's output to be x.w + b for every
# training example x i.e the output is the sum of 
# dot product of the weight vector with the training input vector 
# and the bias

# We can bulk compute the dot products of all the training
# examples with a given weight vector by just multiplying 
# the matrix X (whose rows correspond to individual training
# examples) with vector w.
# Finally we add the bias vector. b = 5.0.
# Note that X_mul_w is a vector, whereas b is a scalar.
# In this case, the scalar is broadcasted
# to all elements of the vector.
#
# Let us reload the cat-brain data matrix X.
X = torch.tensor([[0.11, 0.09], [0.01, 0.02], [0.98, 0.91], [0.12, 0.21],
              [0.98, 0.99], [0.85, 0.87], [0.03, 0.14], [0.55, 0.45],
              [0.49, 0.51], [0.99, 0.01], [0.02, 0.89], [0.31, 0.47],
              [0.55, 0.29], [0.87, 0.76], [0.63, 0.24]])
w = torch.rand((2, 1)) # a randomly initialized weight vector
b = 5.0                      # random bias value
X_mul_w = torch.matmul(X, w)

# Given the random weight vector and bias, the model will output the
# vector model_out (of course this will be very different from the
# desired output, we have not chosen the weights and bias optimally.
# How to choose optimal will be shown later).
model_output = X_mul_w + b

print("Shape of Xw: {}\nmodel output:\n{}".format(X_mul_w.shape,
                                                  model_output))

Shape of Xw: torch.Size([15, 1])
model output:
tensor([[5.1794],
        [5.0271],
        [6.6973],
        [5.2980],
        [6.7705],
        [6.5460],
        [5.1546],
        [5.8970],
        [5.8990],
        [5.8826],
        [5.8320],
        [5.7036],
        [5.7506],
        [6.4630],
        [5.7754]])


#### Matrix Matrix Multiplication
Consider a matrix $A_{m,p}$ with m rows and p columns.
<br>Let us multiply it with another matrix $B_{p,n}$ with p rows and n columns.
<br>The resultant matrix $C_{m,n}$ will contain m rows and n columns.
<br>Note that number of columns in the left matrix $A$ should match the number of
<br>rows in the right matrix $B$.

\begin{align*}
\begin{bmatrix}
        c_{11} & c_{12} \\
        c_{21} & c_{22} \\
        c_{31} & c_{32}
\end{bmatrix}
& = 
\begin{bmatrix}
        a_{11} & a_{12} \\
        a_{21} & a_{22} \\
        a_{31} & a_{32}
\end{bmatrix}
\begin{bmatrix}
        b_{11} & b_{12} \\
        b_{21} & b_{22} \\
\end{bmatrix}\\
\\
c_{11} &= a_{11}b_{11} + a_{12}b_{21} \\
c_{12} &= a_{11}b_{12} + a_{12}b_{22} \\
c_{21} &= a_{21}b_{11} + a_{22}b_{21} \\
c_{22} &= a_{21}b_{12} + a_{22}b_{22} \\
c_{31} &= a_{31}b_{11} + a_{32}b_{21} \\
c_{32} &= a_{31}b_{12} + a_{32}b_{22}
\end{align*}
<br>
<br>
In general
$$
\\
c_{ij} = \sum_{i=1}^p a_{ip}b_{pj}
$$

In [6]:
A = torch.tensor([[1, 2], [3, 4], [5, 6]])
B = torch.tensor([[7, 8], [9, 10]])
C = torch.matmul(A, B)
print("A\n{}\n".format(A))
print("B\n{}\n".format(B))
print("C\n{}\n".format(C))

# Dot product is a special case of matrix multiplication
w = torch.tensor([1, 2, 3])
x = torch.tensor([4, 5, 6])
assert torch.dot(w, x) == torch.matmul(w.t(), x)

A
tensor([[1, 2],
        [3, 4],
        [5, 6]])

B
tensor([[ 7,  8],
        [ 9, 10]])

C
tensor([[ 25,  28],
        [ 57,  64],
        [ 89, 100]])



#### Transpose of Matrix Product
Given two matrices $A$ and $B$ where the number of columns of $A$ matches
<br>the number of rows of $B$, the transpose of their product is the product
<br>of the individual transposes in reversed order. $(AB)^T = B^{T}A^{T}$

In [7]:
print("Transpose of the product")
print(torch.matmul(A, B).T) 
print("Product of individual transposes in reverse order")
print(torch.matmul(B.T, A.T))

assert torch.all(torch.matmul(A, B).T == torch.matmul(B.T, A.T))

# This applies to matrix vector multiplication as well
assert torch.all(torch.matmul(A.T, x).T == torch.matmul(x.T, A))

Transpose of the product
tensor([[ 25,  57,  89],
        [ 28,  64, 100]])
Product of individual transposes in reverse order
tensor([[ 25,  57,  89],
        [ 28,  64, 100]])


/var/folders/tx/pqtg9xkd5156pd7hbgdr_4140000gn/T/ipykernel_32801/1962590849.py:9: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3281.)
  assert torch.all(torch.matmul(A.T, x).T == torch.matmul(x.T, A))


## Matrix inverse

Let us say we want to solve a simultaneuous equation with two variables $x_1$ and $x_2$,
<br>Such an equation can be written as
\begin{align*}
a_{11}x_1+ a_{12}x_2 &= b_1 \\
a_{21}x_1 + a_{22}x_2 &= b_2
\end{align*}
This can be written using matrices and vectors as
$$
A\vec{x} = \vec{b}
$$ where
$$
A =
\begin{bmatrix}
        a_{11} & a_{12} \\
        a_{21} & a_{22} \\
\end{bmatrix}
\space \space \space
\vec{x} = 
\begin{bmatrix}
        x_{1} \\
        x_{2} \\
\end{bmatrix}
\space \space \space
\vec{b} = 
\begin{bmatrix}
        b_{1} \\
        b_{2} \\
\end{bmatrix}
$$
Solution of $A\vec{x} = \vec{b}$ is 
$$
\vec{x} = A^{-1}\vec{b}
$$
where $A^{-1}$ is the matrix inverse, (assumed $det(A) \neq 0$).
<br>Compare this with the scalar equation $ax = b$ whose solution is $x = a^{-1}b$.

The determinant can be computed as  $$det(A) = a_{11}a_{22} - a_{12}a_{21} $$ 
The inverse is
$$
A^{-1} = \frac{1}{det(A)}
\begin{bmatrix}
        a_{22} & -a_{12} \\
        -a_{21} & a_{11}
\end{bmatrix}
$$
Although the above example is shown with a small $2\times$ system ofsimultaneous equations,
<br> the code below is general and works for arbitrary sized linear systems.

In [8]:
def determinant(A):
    return torch.linalg.det(A)

def inverse(A):
    return torch.linalg.inv(A)

# Case1: Invertible Matrix
A = torch.tensor([[2, 3], 
              [2, 2]], dtype=torch.float)

A_inv = inverse(A)
I = torch.matmul(A, A_inv)

# We assert that A A_inv is identity matrix

assert torch.allclose(I, torch.eye(2), atol=1e-5) #torch.eye is used to generate identity matrix

print("Invertible Matrix")
print("Determinant: {}".format(determinant(A))) # 2*2 - 2*3 = -2
print("A:\n{}\n".format(A)) 
print("A Inverse:\n{}\n".format(A_inv))
print("Note that determinant of A is {} (non-zero),\n"
      "hence A is invertible and hence the equation is solvable\n".\
      format(determinant(A)))
print("Matmul(A, A_inv) is \n {},"
      "which is the identity matrix\n\n".format(I))

Invertible Matrix
Determinant: -1.9999998807907104
A:
tensor([[2., 3.],
        [2., 2.]])

A Inverse:
tensor([[-1.0000,  1.5000],
        [ 1.0000, -1.0000]])

Note that determinant of A is -1.9999998807907104 (non-zero),
hence A is invertible and hence the equation is solvable

Matmul(A, A_inv) is 
 tensor([[ 1.0000e+00, -2.3842e-07],
        [ 0.0000e+00,  1.0000e+00]]),which is the identity matrix




In [9]:
# The torch function torch.eye returns an Identity matrix.
I = torch.eye(2)
# IA = AI = A
assert torch.allclose(torch.matmul(I, A), A, atol=1e-5)\
       and torch.allclose(A, torch.matmul(A,I), atol=1e-5)

# AA_inv = A_invA = I
assert torch.allclose(torch.matmul(A, A_inv), I, atol=1e-5)\
       and torch.allclose(torch.matmul(A_inv, A), I, atol=1e-5)

# aI = Ia = a
a = A[0, :]
assert torch.allclose(torch.matmul(a, I), a, atol=1e-5) and torch.allclose(torch.matmul(I, a.T), a, atol=1e-5)

In [10]:
# Case2: Singular Matrix - attempt to invert it directly causes an exception.
B = torch.tensor([[1, 1], [2, 2]], dtype=torch.float)
print("Non-Invertible Matrix")
print("Determinant: {}\n".format(determinant(B)))
try:
    B_inv = inverse(B)
except RuntimeError as e:
    print("B {}\ncannot be inverted\n"
          "because it is a {}\n".format(B, e))
print("Note that determinant of B is {}\n"
      "hence B is non-invertible".format(determinant(B)))

Non-Invertible Matrix
Determinant: 0.0

B tensor([[1., 1.],
        [2., 2.]])
cannot be inverted
because it is a linalg.inv: The diagonal element 2 is zero, the inversion could not be completed because the input matrix is singular.

Note that determinant of B is 0.0
hence B is non-invertible
